In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2024-02-13 10:23:58--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: 'source.zip'

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2024-02-13 10:23:59 (606 MB/s) - 'source.zip' saved [2707/2707]



In [4]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


In [14]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [15]:
log_file = "log_file.txt"
target_file = "transformed_file.csv"

In [30]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [34]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

In [36]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name","height","weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name,"height":height, "weight":weight}])], ignore_index=True)
    return dataframe

In [21]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height', 'weight'])
    
    for csvfile in glob.glob("*.csv"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
        
        
    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True)
        
    for xmlfile in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True)
    
    
    return extracted_data

In [22]:
def transform(data):
    
    data['height'] = round(data.height * 0.0254,2)
    
    
    data['weight'] = round(data.weight * 0.45359237,2)
    
    
    return data

In [41]:
def load_data(target_file, transformed_data):
    transformed_data.to_csv(target_file)

In [25]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' 
    now = datetime.now() 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

In [27]:
log_progress("ETL Started")

In [37]:
log_progress("Extraction Started")
extracted_data = extract()

/tmp/ipykernel_33/3665740089.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
/tmp/ipykernel_33/1084420215.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name,"height":height, "weight":weight}])], ignore_index=True)
/tmp/ipykernel_33/1084420215.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entr

In [38]:
log_progress("Extract Ended")

In [39]:
log_progress("Transform Phase Started")
transformed_data = transform(extracted_data)
print("Tranformed Ended")
print(transformed_data)

Tranformed Ended
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15   jack    1.74   55.93
16    tom    1.77   64.18
17  tracy    1.78   61.90
18   john    1.72   50.97
19   jack    1.74   55.93
20    tom    1.77   64.18
21  tracy    1.78   61.90
22   john    1.72   50.97
23   jack    1.74   55.93
24    tom    1.77   64.18
25  tracy    1.78   61.90
26   john    1.72   50.97
27  simon    1.72   50.97
28  jacob    1.70   54.73
29  cindy    1.69   57.81
30   ivan    1.72   51.77
31  simon    1.72   50.97
32  jacob    1.70   54.73
33  cindy    1.69   57.81
34   ivan    1.72   51.77
35  simon    1.72   50.97
36  jacob    1.70   5

In [42]:
log_progress("Load Started")
load_data(target_file, transformed_data)

In [43]:
log_progress("ETL Job Ended")
print(transformed_data)

     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15   jack    1.74   55.93
16    tom    1.77   64.18
17  tracy    1.78   61.90
18   john    1.72   50.97
19   jack    1.74   55.93
20    tom    1.77   64.18
21  tracy    1.78   61.90
22   john    1.72   50.97
23   jack    1.74   55.93
24    tom    1.77   64.18
25  tracy    1.78   61.90
26   john    1.72   50.97
27  simon    1.72   50.97
28  jacob    1.70   54.73
29  cindy    1.69   57.81
30   ivan    1.72   51.77
31  simon    1.72   50.97
32  jacob    1.70   54.73
33  cindy    1.69   57.81
34   ivan    1.72   51.77
35  simon    1.72   50.97
36  jacob    1.70   54.73
37  cindy   